In [ ]:
from numpy.core.fromnumeric import shape
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from numpy.lib.function_base import average
from re import U
from sklearn import tree
from sklearn.model_selection import train_test_split
import pydot
import graphviz
import pandas as pd
import pydotplus
import collections
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import validation_curve


from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.compose import ColumnTransformer



from sklearn.linear_model import LogisticRegression
from sklearn import linear_model as linmod
from sklearn import metrics
from sklearn import preprocessing as preproc
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score




dataFrame = pd.read_excel('shorted_revised_dataset.xlsx')


y=dataFrame.IsIncomplete


x=dataFrame.drop(["IsIncomplete"],axis=1)

#print(x)

labelencode_x= LabelEncoder()

x.GameDate=labelencode_x.fit_transform(x.GameDate)




#X = [[0, 'a'], [0, 'b'], [1, 'a'], [2, 'b']]

#ct = ColumnTransformer(
 #   [('one_hot_encoder', OneHotEncoder(categories='auto'), [4])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
                                                                    # Leave the rest of the columns untouched
#                       )

#x = ct.fit_transform(x)

#onehot=OneHotEncoder(categories=[5,11,14])


#z= onehot.fit_transform(x).toarray()


e=pd.get_dummies(x.OffenseTeam)

k=pd.get_dummies(x.Formation)

n=pd.get_dummies(x.PassType)

merged_dataframe= pd.concat([x,e,k,n],axis='columns')




shorted_merge_dataframe=merged_dataframe.drop(["OffenseTeam","Formation","PassType"],axis=1)


scalerX = MinMaxScaler()
scalerX.fit(shorted_merge_dataframe)
shorted_merge_dataframe = scalerX.transform(shorted_merge_dataframe)


#print(shorted_merge_dataframe)


#shorted_merge_dataframe.to_excel("output.xlsx")


#shorted_normalized = scalerX.transform(shorted_merge_dataframe)


X_train, X_test, y_train, y_test = train_test_split(shorted_merge_dataframe, y, test_size=0.3, random_state=42)

#scaler=MinMaxScaler()

#X_train=scaler.fit_transform(X_train)

#X_test=scaler.transform(X_test)


#print(X_train)


mlr = linmod.LogisticRegression(max_iter=1000,tol=1e-6)     # creates the regressor object – note the lower tolerance!

mlr.fit(X_train, y_train )

Ypred = mlr.predict(X_test)

#Ypredclass = 1*(Ypred > 0.5)

print("R2 = %f,  MSE = %f,  Classification Accuracy = %f" % (metrics.r2_score(y_test, Ypred), metrics.mean_squared_error(y_test, Ypred), metrics.accuracy_score(y_test, Ypred)))

print("W: ", np.append(np.array(mlr.intercept_), mlr.coef_))

print("This is the shape of the X_test")
print(shape(X_test))

normpred=shorted_merge_dataframe

print(type(normpred))

target=y

print(type(target))

def tryVariableSelection(pred, targ, sel, dir, labels):
    ranseed = 42
    xtrain, xtest, ytrain, ytest = train_test_split(pred, targ, test_size=0.3, random_state=ranseed)
    model = linmod.LogisticRegression(max_iter=1000,tol=1e-6)
    if sel == 'sequential':
        selector = SequentialFeatureSelector(model, direction=dir, n_features_to_select=6)
    elif sel == 'RFE':
        selector = RFE(model, step=1, n_features_to_select=6)
    elif sel == 'RFECV':
        selector = RFECV(model, step=1, cv=5)
    selector.fit(xtrain, ytrain)
    newxtrain = selector.transform(xtrain)
    newxtest = selector.transform(xtest)
    model.fit(newxtrain, ytrain)
    print("This is the shape of the newxtest ")
    print(shape(newxtest))
    print(newxtrain)
    Ypred=model.predict(newxtest)
   # print("\nUsing: {0}".format(labels[selector.get_support() == True]))
    print("Method {0}: Testing set R-sq={1:8.5f},  test set MSE={2:e}".format(dir, metrics.r2_score(ytest,model.predict(newxtest) ),mean_squared_error(ytest, model.predict(newxtest)) ))
    print ("Classification accuracy = %f" % metrics.accuracy_score(y_test, Ypred))

featureLabels=normpred

tryVariableSelection(normpred, target, 'sequential', 'forward', featureLabels)
#tryVariableSelection(normpred, target, 'sequential', 'backward', featureLabels)
#tryVariableSelection(normpred, target, 'RFE', 'RFE', featureLabels)
#tryVariableSelection(normpred, target, 'RFECV', 'RFECV', featureLabels)




#X_train=scaler.fit_transform(X_train)

#X_test=scaler.transform(X_test)


R2 = 0.683468,  MSE = 0.071580,  Classification Accuracy = 0.928420
W:  [ 4.06376806e+00  1.22382561e-01  9.36855724e-02 -8.19855851e-02
  2.22321691e-03  6.16549968e-01 -2.43408597e+00  4.44417480e-01
 -2.85296436e+00 -2.86806244e+01 -1.14431231e+00 -1.84210116e-01
 -2.29220625e-02 -1.05542888e-01  3.00351778e-02 -1.74579234e-01
 -2.47981968e-01  3.08360449e-02  3.42769282e-01  1.13004737e-01
  2.73896687e-01  5.30870965e-02 -1.21316297e-01 -1.89105286e-01
 -6.47188315e-02 -3.68407187e-02  6.68408416e-02 -1.20669146e-01
 -3.95252180e-03 -2.23921181e-02 -5.43463403e-02 -3.81186168e-02
  6.06644789e-02 -1.50192870e-01  1.32033189e-01  1.33100352e-01
  2.19116578e-01 -7.51615964e-02 -1.62527250e-01  1.71670847e-01
  7.35484953e-02  2.69592851e-01 -1.89556419e-01  2.50203340e-01
 -1.35395701e-01 -1.21196457e-01  1.24511954e-02  1.74989993e+00
  1.10074540e+00  1.64212828e+00 -1.75736338e+00  0.00000000e+00
 -6.80071500e-01  2.25047940e-01  2.70135999e-01 -9.70053582e-01
 -6.62627064e-01 -